In [13]:
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import geojson

# Load GeoJSON file from Github
# Author: Francesco Schwarz
# source: https://github.com/isellsoap/deutschlandGeoJSON
with open('2_hoch.geo.json') as b:
    bundeslaender = geojson.load(b)

In [14]:
# bundeslaender

In [15]:
##### Incorporate data
# Convert 'Date' from object to Date
df = pd.read_csv('group_rossmann_dataprep.csv', sep=';')
df.head()


Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0    103         d          c               5210.0                        5.0   
1    894         a          a                190.0                       11.0   
2    972         a          a              14960.0                        NaN   
3   1081         b          a                400.0                        3.0   
4    665         a          a                 90.0                       12.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2015.0       0              NaN              NaN   
1                    2012.0       0              NaN              NaN   
2                       NaN       0              NaN              NaN   
3                    2006.0       0              NaN              NaN   
4                    2012.0       1             14.0           2011.0   

     PromoInterval State           StateName  DayOfWeek        Date  Sales  \
0              NaN    BE              Berlin          1  10.06.2013      0   
1              NaN    NW  NordrheinWestfalen          1  10.06.2013      0   
2              NaN    ST       SachsenAnhalt          1  10.06.2013      0   
3              NaN    BE              Berlin          1  10.06.2013      0   
4  Jan,Apr,Jul,Oct    HE              Hessen          1  24.06.2013      0   

   Customers  Open  Promo  StateHoliday  SchoolHoliday  
0          0     0      0           0.0              0  
1          0     0      0           0.0              0  
2          0     0      0           0.0              0  
3          0     0      0           0.0              0  
4          0     0      0           0.0              0

In [16]:
df.dtypes

Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
State                         object
StateName                     object
DayOfWeek                      int64
Date                          object
Sales                          int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                 float64
SchoolHoliday                  int64
dtype: object

In [17]:
# Convert Column "Date" to DateTime
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y")
df.dtypes

Store                                 int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
State                                object
StateName                            object
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                        float64
SchoolHoliday                         int64
dtype: object

In [18]:
df.head()

Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0    103         d          c               5210.0                        5.0   
1    894         a          a                190.0                       11.0   
2    972         a          a              14960.0                        NaN   
3   1081         b          a                400.0                        3.0   
4    665         a          a                 90.0                       12.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2015.0       0              NaN              NaN   
1                    2012.0       0              NaN              NaN   
2                       NaN       0              NaN              NaN   
3                    2006.0       0              NaN              NaN   
4                    2012.0       1             14.0           2011.0   

     PromoInterval State           StateName  DayOfWeek       Date  Sales  \
0              NaN    BE              Berlin          1 2013-06-10      0   
1              NaN    NW  NordrheinWestfalen          1 2013-06-10      0   
2              NaN    ST       SachsenAnhalt          1 2013-06-10      0   
3              NaN    BE              Berlin          1 2013-06-10      0   
4  Jan,Apr,Jul,Oct    HE              Hessen          1 2013-06-24      0   

   Customers  Open  Promo  StateHoliday  SchoolHoliday  
0          0     0      0           0.0              0  
1          0     0      0           0.0              0  
2          0     0      0           0.0              0  
3          0     0      0           0.0              0  
4          0     0      0           0.0              0

In [19]:
# Group By Date and StateName, aggregate by sum of Sales -> nyc.groupby (....).agg(....)
# check results
df_sales=df.groupby(["State", "StateName"], as_index=False).agg({"Sales": "sum"})
df_sales

State           StateName       Sales
0     BE              Berlin   604709903
1     BW   BadenWuerttemberg   355745661
2     BY              Bayern   742361827
3     HE              Hessen   603605998
4     HH             Hamburg   169053366
5     NW  NordrheinWestfalen  1598936483
6     RP      RheinlandPfalz   181243703
7     SH   SchleswigHolstein   618426981
8     SN             Sachsen   402442109
9     ST       SachsenAnhalt   276915114
10    TH          Thueringen   188595349

In [20]:
# Add map of Germany
fig = px.choropleth_mapbox(data_frame = df_sales, 
                           geojson = bundeslaender, 
                           featureidkey = 'properties.id', 
                           locations = 'State', 
                           color = 'Sales',
                           hover_name = 'StateName',
                           color_continuous_scale = "Teal",
                           range_color = (200000000, 1800000000),
                           mapbox_style = "carto-positron",
                           zoom = 4.5, 
                           center = {"lat": 51.165691, "lon": 10.451526},
                           opacity = 0.8,
                           labels = {'Sales':'Sales rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(marker_line_width = 0.3, marker_line_color = 'black')
fig.show()